In [581]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout,GRU
from keras.optimizers import Adam, RMSprop
from keras.losses import binary_crossentropy
from keras.metrics import accuracy

In [582]:
data_df = pd.read_csv('C:\\Users\\hit42\\Desktop\\2020 CUAI\\공모전\\preprocessed_data_final.csv')
data_df

,날짜,지수종가,종가이동평균_5,거래량,거래량이동평균_5,자본금,전체종목수,상승종목수,하락종목수,보합종목수,...,운수장비,서비스업,snp500,nasdaq,shanghai,COMP,NEG,NEUT,None,POS
0,2001-01-08,586.65,553.580,558645100,19.873264,84853590,902,533,278,45,...,9.22,11.03,0.77,-0.49,-1.09,0.005082,0.360881,0.041947,0.000518,0.591572
1,2001-01-09,589.92,567.374,558967700,20.016030,84881174,902,527,282,49,...,9.34,11.32,-0.27,1.89,-0.04,0.011277,0.478814,0.085582,0.016079,0.408248
2,2001-01-10,560.81,575.250,642378200,20.137292,84891369,902,137,698,23,...,8.64,11.08,1.76,3.39,1.16,0.000000,0.555556,0.055556,0.000000,0.388889
3,2001-01-11,561.79,576.004,518788600,20.163623,84906240,902,574,242,43,...,8.41,11.29,0.10,4.61,-0.30,0.007067,0.381705,0.140920,0.045477,0.424830
4,2001-01-12,587.87,577.408,713787700,20.209960,84909240,902,494,298,68,...,8.95,11.83,-0.19,-0.53,-0.68,0.003084,0.369053,0.021311,0.000881,0.605672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4440,2018-12-21,2061.49,2066.730,311388800,19.819547,107914395,900,381,445,70,...,53.68,25.44,-2.62,-2.99,-0.79,0.022321,0.358543,0.064166,0.037088,0.517881
4441,2018-12-24,2055.01,2063.514,285275000,19.737655,107972455,900,285,538,71,...,53.59,25.35,-2.64,-2.21,0.43,0.018742,0.354360,0.057304,0.048192,0.521403
4442,2018-12-26,2028.01,2056.694,321499300,19.678019,108073277,900,128,724,44,...,52.15,24.78,5.05,5.84,-0.26,0.020903,0.346296,0.063338,0.043228,0.526235
4443,2018-12-27,2028.44,2046.614,398021300,19.666131,108148532,901,521,331,45,...,52.35,25.16,0.77,0.38,-0.61,0.016228,0.354153,0.059219,0.045312,0.525088


In [583]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4445 entries, 0 to 4444
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   날짜         4445 non-null   object 
 1   지수종가       4445 non-null   float64
 2   종가이동평균_5   4445 non-null   float64
 3   거래량        4445 non-null   int64  
 4   거래량이동평균_5  4445 non-null   float64
 5   자본금        4445 non-null   int64  
 6   전체종목수      4445 non-null   int64  
 7   상승종목수      4445 non-null   int64  
 8   하락종목수      4445 non-null   int64  
 9   보합종목수      4445 non-null   int64  
 10  상한종목수      4445 non-null   int64  
 11  하한종목수      4445 non-null   int64  
 12  연중최고가종목수   4445 non-null   int64  
 13  연중최저가종목수   4445 non-null   int64  
 14  52주신고가종목수  4445 non-null   int64  
 15  52주신저가종목수  4445 non-null   int64  
 16  변동률        4445 non-null   float64
 17  금괴(일간)     4445 non-null   float64
 18  국고채권(10년)  4445 non-null   float64
 19  원유 WTI     4445 non-null   float64
 20  USD     

In [584]:
data_df.describe()

,지수종가,종가이동평균_5,거래량,거래량이동평균_5,자본금,전체종목수,상승종목수,하락종목수,보합종목수,상한종목수,...,운수장비,서비스업,snp500,nasdaq,shanghai,COMP,NEG,NEUT,None,POS
count,4445.000000,4445.000000,4.445000e+03,4445.000000,4.445000e+03,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,...,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000
mean,1566.345773,1565.679890,4.218820e+08,19.795263,9.360336e+07,893.698538,383.814398,408.103712,70.534083,9.534758,...,11.993291,16.353618,0.020333,0.028823,0.011417,0.019328,0.377203,0.062947,0.041461,0.499060
std,564.712688,564.783055,1.889785e+08,0.346692,9.213751e+06,30.065657,129.055575,133.096115,17.167047,10.934980,...,5.949642,8.228201,1.183392,1.416666,1.552455,0.007372,0.043241,0.018128,0.024389,0.045411
min,468.760000,474.372000,1.363290e+08,18.900436,7.858320e+07,832.000000,15.000000,46.000000,2.000000,0.000000,...,5.300000,1.590000,-9.840000,-9.140000,-8.840000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1016.820000,1010.090000,3.056365e+08,19.567431,8.409615e+07,878.000000,301.000000,317.000000,60.000000,3.000000,...,8.700000,11.580000,-0.430000,-0.560000,-0.650000,0.017433,0.363337,0.058086,0.038783,0.485093
50%,1766.710000,1763.866000,3.813070e+08,19.761609,9.764014e+07,893.000000,385.000000,396.000000,72.000000,7.000000,...,10.040000,14.680000,0.040000,0.050000,0.000000,0.019940,0.375458,0.061697,0.042522,0.499511
75%,1996.980000,1996.006000,4.864097e+08,19.992901,1.004792e+08,919.000000,468.000000,490.000000,83.000000,13.000000,...,13.180000,20.440000,0.540000,0.690000,0.710000,0.022094,0.388546,0.065362,0.045991,0.512412
max,2598.190000,2575.138000,2.379294e+09,21.391700,1.106207e+08,1520.000000,955.000000,865.000000,122.000000,376.000000,...,54.520000,40.890000,14.520000,11.810000,9.860000,0.166667,0.798948,0.424242,1.000000,1.000000


In [585]:
data_df = data_df.drop('날짜', axis=1, inplace=False)

In [586]:
data_df

,지수종가,종가이동평균_5,거래량,거래량이동평균_5,자본금,전체종목수,상승종목수,하락종목수,보합종목수,상한종목수,...,운수장비,서비스업,snp500,nasdaq,shanghai,COMP,NEG,NEUT,None,POS
0,586.65,553.580,558645100,19.873264,84853590,902,533,278,45,72,...,9.22,11.03,0.77,-0.49,-1.09,0.005082,0.360881,0.041947,0.000518,0.591572
1,589.92,567.374,558967700,20.016030,84881174,902,527,282,49,64,...,9.34,11.32,-0.27,1.89,-0.04,0.011277,0.478814,0.085582,0.016079,0.408248
2,560.81,575.250,642378200,20.137292,84891369,902,137,698,23,36,...,8.64,11.08,1.76,3.39,1.16,0.000000,0.555556,0.055556,0.000000,0.388889
3,561.79,576.004,518788600,20.163623,84906240,902,574,242,43,87,...,8.41,11.29,0.10,4.61,-0.30,0.007067,0.381705,0.140920,0.045477,0.424830
4,587.87,577.408,713787700,20.209960,84909240,902,494,298,68,27,...,8.95,11.83,-0.19,-0.53,-0.68,0.003084,0.369053,0.021311,0.000881,0.605672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4440,2061.49,2066.730,311388800,19.819547,107914395,900,381,445,70,0,...,53.68,25.44,-2.62,-2.99,-0.79,0.022321,0.358543,0.064166,0.037088,0.517881
4441,2055.01,2063.514,285275000,19.737655,107972455,900,285,538,71,1,...,53.59,25.35,-2.64,-2.21,0.43,0.018742,0.354360,0.057304,0.048192,0.521403
4442,2028.01,2056.694,321499300,19.678019,108073277,900,128,724,44,0,...,52.15,24.78,5.05,5.84,-0.26,0.020903,0.346296,0.063338,0.043228,0.526235
4443,2028.44,2046.614,398021300,19.666131,108148532,901,521,331,45,1,...,52.35,25.16,0.77,0.38,-0.61,0.016228,0.354153,0.059219,0.045312,0.525088


In [587]:
def normalize_data(df):
    scaler = MinMaxScaler()
    for i, column in enumerate(data_df.columns):
        df[data_df.columns[i]] = scaler.fit_transform(df[data_df.columns[i]].values.reshape(-1,1))
    return df

In [588]:
normalized_data_df = normalize_data(data_df)
normalized_data_df.describe()

,지수종가,종가이동평균_5,거래량,거래량이동평균_5,자본금,전체종목수,상승종목수,하락종목수,보합종목수,상한종목수,...,운수장비,서비스업,snp500,nasdaq,shanghai,COMP,NEG,NEUT,None,POS
count,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,...,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000,4445.000000
mean,0.515436,0.519481,0.127310,0.359186,0.468830,0.089678,0.392356,0.442129,0.571117,0.025358,...,0.135987,0.375665,0.404776,0.437653,0.473338,0.115969,0.472125,0.148376,0.041461,0.499060
std,0.265194,0.268846,0.084254,0.139163,0.287593,0.043700,0.137293,0.162511,0.143059,0.029082,...,0.120879,0.209369,0.048579,0.067621,0.083019,0.044230,0.054123,0.042730,0.024389,0.045411
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.257374,0.255011,0.075484,0.267734,0.172078,0.066860,0.304255,0.330891,0.483333,0.007979,...,0.069078,0.254198,0.386289,0.409547,0.437968,0.104597,0.454770,0.136916,0.038783,0.485093
50%,0.609529,0.613821,0.109221,0.345677,0.594832,0.088663,0.393617,0.427350,0.583333,0.018617,...,0.096302,0.333079,0.405583,0.438663,0.472727,0.119640,0.469941,0.145428,0.042522,0.499511
75%,0.717666,0.724323,0.156079,0.438518,0.683448,0.126453,0.481915,0.542125,0.675000,0.034574,...,0.160098,0.479644,0.426108,0.469212,0.510695,0.132564,0.486322,0.154067,0.045991,0.512412
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [589]:
def generator(data, lookback=10, delay=5, min_index=0, max_index=None):
    data = data.values #DataFrame을 넘파이로 바꾸기
    max_index = len(data) - delay -1 #4439
    i = min_index + lookback #index10부터 
    samples = np.zeros((len(range(max_index-i+1)), lookback, data.shape[-1])) #(4430, 10, 33)
    targets = np.zeros((len(range(max_index-i+1)),)) #(4430,)
    while i <= max_index:
        indices = range(i-lookback, i)
        samples[i-lookback] = data[indices]
        if data[i+delay][0] - data[i][0] >= 0:
            targets[i-lookback] = 1
        else:
            targets[i-lookback] = 0
        i+=1
    return samples, targets

In [590]:
samples, targets = generator(normalized_data_df, min_index=0, max_index=3500)


In [591]:
print(samples.shape)
print(targets.shape) 

(4430, 10, 33)
(4430,)


In [592]:
def split_data(sample, target, train_size_percentage=90, shuffle=True):
    if shuffle is True:
        index = np.arange(sample.shape[0])
        np.random.shuffle(index)
        sample = sample[index]
        target = target[index]
    train_size = train_size_percentage*sample.shape[0]//100
    test_size = sample.shape[0] - train_size
    
    x_train = sample[:train_size]
    y_train = target[:train_size]
    x_test = sample[train_size:]
    y_test = target[train_size:]
    
    return x_train, y_train, x_test, y_test    

In [593]:
x_train, y_train, x_test, y_test = split_data(samples, targets)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3987, 10, 33)
(3987,)
(443, 10, 33)
(443,)


In [594]:
d= 0.5
model = Sequential()
model.add(GRU(32,input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(d))
model.add(GRU(64))
model.add(Dropout(d))
model.add(Dense(32, activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam')

model.fit(x_train,y_train,batch_size=32,epochs=100,verbose=1,validation_split=0.2)

Train on 3189 samples, validate on 798 samples
Epoch 1/100
3189/3189 [==============================] - 12s 4ms/step - loss: 0.6896 - val_loss: 0.6771
Epoch 2/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.6776 - val_loss: 0.6693
Epoch 3/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.6746 - val_loss: 0.6591
Epoch 4/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.6602 - val_loss: 0.6570
Epoch 5/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.6493 - val_loss: 0.6374
Epoch 6/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.6345 - val_loss: 0.6107
Epoch 7/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.6096 - val_loss: 0.5699
Epoch 8/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.5814 - val_loss: 0.5419
Epoch 9/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.5604 - val_loss: 0.5236
Epoch 10/100
3189/3189 [=

3189/3189 [==============================] - 4s 1ms/step - loss: 0.3506 - val_loss: 0.3948
Epoch 80/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3599 - val_loss: 0.3914
Epoch 81/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3625 - val_loss: 0.3954
Epoch 82/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3503 - val_loss: 0.3929
Epoch 83/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3546 - val_loss: 0.4180
Epoch 84/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3512 - val_loss: 0.3918
Epoch 85/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3457 - val_loss: 0.4266
Epoch 86/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3582 - val_loss: 0.4032
Epoch 87/100
3189/3189 [==============================] - 4s 1ms/step - loss: 0.3364 - val_loss: 0.3981
Epoch 88/100
3189/3189 [==============================] - 4s 1ms/step - loss:

In [595]:
def eval_acc(models, x_test_data, y_test_data, threshold=0.5):
    scores = models.predict(x_test_data)
    classified_scores=[]
    for score in scores:
        if score >= threshold:
            classified_scores.append(1)
        else:
            classified_scores.append(0)
    
    acc=0
    for i, classified_score in enumerate(classified_scores):
        if classified_score== y_test_data[i]:
            acc+=1
    accuracy_score = acc/len(y_test_data)*100
    
    return str(accuracy_score) + ' %'


In [596]:
eval_acc(model, x_test, y_test)

'82.39277652370203 %'